<a href="https://colab.research.google.com/github/solcanalla/fiumark/blob/main/svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Modelo: SVM

##Inicialización##

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
import matplotlib
import seaborn as sns
from sklearn.svm import SVC

In [2]:
!git clone https://github.com/solcanalla/fiumark.git
%cd fiumark
!git pull origin main
import preprocessing as pp

Cloning into 'fiumark'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 104 (delta 54), reused 44 (delta 17), pack-reused 0
Receiving objects: 100% (104/104), 27.31 KiB | 822.00 KiB/s, done.
Resolving deltas: 100% (54/54), done.
/content/fiumark
From https://github.com/solcanalla/fiumark
 * branch            main       -> FETCH_HEAD
Already up to date.


##Pre procesamiento##

In [3]:
pd.options.display.max_columns = None
df = pp.get_dataset()
df = pp.knn_preprocessing(df)
X_train, X_test, y_train, y_test = pp.get_train_test_data(df)

##Entrenamiento##


In [4]:
from sklearn.model_selection import GridSearchCV

params = {
    'C': np.arange(1, 5),
    'kernel': ['linear', 'poly','rbf'],
    'degree': np.arange(1, 5),
    'gamma':np.arange(1, 3),
    'coef0': np.arange(1, 2)
}

clf = SVC()

gscv = GridSearchCV(
    clf, params, scoring='roc_auc', n_jobs=-1, cv=5, return_train_score=True
).fit(X_train, y_train)

In [5]:
print(f"Best score: {gscv.best_score_}")
print(f"Best params {gscv.best_params_}")

Best score: 0.8409707197752517
Best params {'C': 4, 'coef0': 1, 'degree': 1, 'gamma': 2, 'kernel': 'poly'}


##Evaluación del modelo##

In [6]:
y_pred = gscv.predict(X_test)

###Precision recall y F1-score###

In [7]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.84      0.92      0.87        95
         1.0       0.86      0.74      0.80        66

    accuracy                           0.84       161
   macro avg       0.85      0.83      0.84       161
weighted avg       0.85      0.84      0.84       161



##Accuracy###

In [9]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.84472049689441

###Auc-Roc###

In [10]:
from sklearn.metrics import roc_auc_score
auc_roc = roc_auc_score(y_test, y_pred)
auc_roc

0.8291068580542263

##Predicción##

In [8]:
X_holdout = pd.read_csv('https://drive.google.com/uc?export=download&id=1I980-_K9iOucJO26SG5_M8RELOQ5VB6A')
X = pp.knn_preprocessing(X_holdout)
prediction = pd.DataFrame(gscv.predict(X),columns=['volveria'],dtype=int)
pred_formatted = pd.concat([X_holdout.id_usuario,prediction],axis=1)
pred_formatted.to_csv('svm.csv')
pred_formatted

,id_usuario,volveria
0,650,1
1,234,1
2,31,0
3,552,0
4,411,0
...,...,...
85,354,0
86,82,0
87,172,0
88,8,0
